In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import networkx as nx
import random
from networkx import Graph
from sklearn.metrics import mutual_info_score
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from src.pamst import pamst

In [ ]:
print("test")

In [ ]:
# Initialize new random graph
size = 1000
G: Graph = nx.complete_graph(size)
for (u, v, w) in G.edges(data=True):
    w['weight'] = random.random()

In [ ]:
# Compute the Real MST using PRIM
T  = nx.maximum_spanning_tree(G, algorithm ='prim')
weight = 0
for _,_,d in T.edges(data=True):
    weight += d['weight']

pos = nx.circular_layout(G, scale=40)  # positions for all nodes - seed for reproducibility
edge_labels = dict([((u,v,), format_edges(d['weight'])) for u,v,d in G.pamst_edges(data=True)])

# Running PAMST

# Post-Processing Sealfon
